In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
from pathlib import Path
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
fobs_edges = utils.nyquist_freqs_edges()
fobs = 0.5 * (fobs_edges[1:] + fobs_edges[:-1])
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:5.2f}, {fobs[-1]*YR:7.2f}] 1/yr  <==>  [{1/fobs[0]/YR:.2f}, {1/fobs[-1]/YR:.2f}] yr")
print(f"          [{fobs[0]*1e9:5.2f}, {fobs[-1]*1e9:7.2f}] nHz")

In [ ]:
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
gwb = sam.gwb(fobs_edges, realize=200)    # calculate many different realizations

In [ ]:
percs = [2, 5, 10, 25, 50, 75, 90, 95, 98]
gwb_percs = np.percentile(gwb, percs, axis=-1)

In [ ]:
sam._dynamic_binary_number.shape
sam._density.shape

fname = Path(holo._PATH_ROOT).resolve()
fname = fname.joinpath("discretization-comparison_luke-sam-data_2023-02-13.npz")
np.savez(
    fname,
    mtot=sam.mtot, mrat=sam.mrat, redz=sam.redz, fobs=fobs,
    density=sam._density,
    number=sam._dynamic_binary_number,
    gwb=gwb,
    percs=percs,
    gwb_percs=gwb_percs,
)
print(f"Saved to {fname}, size {utils.get_file_size(fname)}")


In [ ]:
number = sam._dynamic_binary_number
nn = number.sum(axis=(0, 1, 2))
plt.loglog(xx, nn)
plt.show()

In [ ]:
nsamp = 10    # number of sample GWB spectra to plot
fig, ax = plot.figax(figsize=[8, 4], xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

# `fobs` are bin centers in CGS units, convert to [1/yr]
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)

# Plot the median GWB spectrum
ax.plot(xx, np.median(gwb, axis=-1), 'k-')

# Plot `nsamp` random spectra 
nsamp = np.min([nsamp, gwb.shape[1]])
idx = np.random.choice(gwb.shape[1], nsamp, replace=False)
# ax.plot(xx, gwb[:, idx], 'k-', lw=1.0, alpha=0.1)

for ii in range(len(percs)//2):
    # print(ii, len(percs)-1-ii, "==>", percs[ii], percs[-1-ii])
    lo = gwb_percs[ii, :]
    hi = gwb_percs[-1-ii, :]
    ax.fill_between(xx, lo, hi, alpha=0.15, color='b')
    
plot._twin_hz(ax)
fname_fig = fname.with_suffix('.png')
fig.savefig(fname_fig, dpi=400)
print(f"Saved to {fname_fig}, size {utils.get_file_size(fname_fig)}")
plt.show()

In [ ]:
nsamp = 10    # number of sample GWB spectra to plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', ylabel='Characteristic Strain $h_c$')

# `fobs` are bin centers in CGS units, convert to [1/yr]
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)

# Plot the median GWB spectrum
ax.plot(xx, np.median(gwb, axis=-1), 'k-')

for ii in range(len(percs)//2):
    print(ii, len(percs)-1-ii, "==>", percs[ii], percs[-1-ii])
    lo = gwb_percs[ii, :]
    hi = gwb_percs[-1-ii, :]
    ax.fill_between(xx, lo, hi, alpha=0.2, color='b')
    
plt.show()